In [4]:
import pandas as pd
import numpy as np
import etna

# 1. Test Classification (Auto-detect should work fine here)
print("=== TEST 1: CLASSIFICATION ===")
model = etna.Model("dummy_data.csv", target="bought")
model.train(epochs=200, lr=0.1)
print(f"✅ Predictions: {model.predict()}")

# 2. Test Regression (Manually specifying task_type="regression")
print("\n=== TEST 2: REGRESSION ===")
# Note: We pass task_type="regression" to skip the auto-detect logic for this tiny dataset
model_reg = etna.Model("dummy_reg.csv", target="salary", task_type="regression")
model_reg.train(epochs=500, lr=0.001) # More epochs for regression accuracy
preds_reg = model_reg.predict()

# Formatting for clean output
print(f"✅ Predictions: {[round(p, 1) for p in preds_reg]}")

=== TEST 1: CLASSIFICATION ===
🔮 Auto-Detected Task: Classification (Target 'bought')
⚙️  Preprocessing data...
🚀 Initializing Rust Core [In: 3, Out: 2]...
🔥 Training started...
✅ Training complete!
⚙️  Transforming input data...
✅ Predictions: [np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1)]

=== TEST 2: REGRESSION ===
🔮 User Task: Regression (Target 'salary')
⚙️  Preprocessing data...
🚀 Initializing Rust Core [In: 2, Out: 1]...
🔥 Training started...
✅ Training complete!
⚙️  Transforming input data...
✅ Predictions: [75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0]
